In [25]:
# Ensemble learning 
'''여러 분류기를 조합하여 사이킷런 투표 기반 분류기를 만들고 훈련'''

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=5000)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)     
# Soft voting을 하기 위해 클래스 확률을 제공하지 않는 SVC 클래스의 매개변수 probability=True로 수정
    # Soft voting은 모든 VotingClassifier의 분류기가 클래스 확률을 예측할 수 있는 경우 가능하다. 
# Hard voting 하는 경우 그대로 진행해도 됨

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf,),('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [26]:
from sklearn.metrics import accuracy_score  # 각 분류기의 정확도 확인
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.893
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


In [27]:
'''하나의 분류기를 사용하지만, 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 방법'''

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=1)  # n_jobs: 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정 
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [ ]:
'''oob 평가'''
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_